In [1]:
import pandas as pd
import os

cwd = os.getcwd()

os.chdir(cwd+'/input csv files')
f1 = open("ratings.csv", 'r')
f2 = open("books.csv", 'r')
f3 = open("book_data_from_kaggle.csv", 'r')
f4 = open("tags.csv", 'r')
f5 = open("book_tags.csv", 'r')

ratings = pd.read_csv(f1)
books = pd.read_csv(f2)
books2 = pd.read_csv(f3)
tags = pd.read_csv(f4)
book_tags = pd.read_csv(f5)

f1.close()
f2.close()
f3.close()
f4.close()
f5.close()

os.chdir(cwd)

In [2]:
# step1: get unique users from ratings.csv (take top 15000 users only)
# agg = aggregate, pd.Series.count on attribute rating count no.of ratings given by that user
useronly = ratings.groupby(by='user_id', as_index=False).agg({'rating': pd.Series.count}).sort_values('rating', ascending=False).head(15000)
print('No.of different users in dataset: '+str(len(useronly)))

No.of different users in dataset: 15000


In [3]:
# step2: remove data related to remaining users from ratings.csv
ratings = ratings[ratings.user_id.isin(useronly.user_id)]

In [4]:
# step3: get unique books from ratings.csv (take top 8000 books only)
bookonly = ratings.groupby(by='book_id', as_index=False).agg({'rating': pd.Series.count}).sort_values('rating', ascending=False).head(8000)
print('No.of different books in dataset: '+str(len(bookonly)))

No.of different books in dataset: 8000


In [5]:
# step4: remove remaining books from books.csv and remaining details from book_tags.csv and ratings.csv
books = books[books.book_id.isin(bookonly.book_id)]
ratings = ratings[ratings.book_id.isin(bookonly.book_id)]
book_tags = book_tags.merge(books[['book_id', 'goodreads_book_id']], how='left', on='goodreads_book_id')
book_tags = book_tags[book_tags.book_id.isin(bookonly.book_id)]
book_tags = book_tags.drop(['book_id'], axis=1)

In [6]:
# step5: add a new column in books.csv ('newbookid') to make all indices continuous because some book_id's are deleted
books = books.reset_index(drop=True)
books['newbookid'] = books.index+1

In [7]:
# step6: add this newbookid column to ratings.csv
ratings = ratings.merge(books[['book_id', 'newbookid']], how='left', on='book_id')

In [8]:
# step7: add a new column in ratings.csv ('newuserid') to make all user_id's continuous because some user_id's are deleted
ratings['newuserid'] = ratings.groupby('user_id').grouper.group_info[0]+1

In [9]:
# step8: add column ('genre' and 'firstgenre') using tags.csv and book_tags.csv
book_tags = book_tags.merge(tags[['tag_id', 'tag_name']], how='left', on='tag_id')
book_tags['tag_name'] = book_tags[['goodreads_book_id', 'tag_name']].groupby(['goodreads_book_id'])['tag_name'].transform(lambda x: '|'.join(x))
book_tags = book_tags.drop(['tag_id', 'count'], axis=1)
book_tags = book_tags[['goodreads_book_id', 'tag_name']].drop_duplicates()
books = books.merge(book_tags[['goodreads_book_id', 'tag_name']], how='left', on='goodreads_book_id')
books = books.rename(columns={'tag_name': 'genre'})
books['genre'] = books['genre'].fillna('Unknown')
books['firstgenre'] = [x.split('|')[0] for x in books['genre']]

In [10]:
# step9: drop columns ('user_id', 'book_id) from ratings.csv
ratings = ratings.drop(['user_id', 'book_id'], axis=1)

In [11]:
print(ratings['rating'].value_counts())

4    757468
5    659929
3    515113
2    134803
1     45375
Name: rating, dtype: int64


In [12]:
# step10: get title,pages,decs,rating columns from book_data_from_kaggle.csv to a temporary df
temp = books2[['book_title', 'book_pages', 'book_desc', 'book_rating']]

In [13]:
# step11: change pattern of string in title of books
#books['title'] = books['title'].str.replace(r"\(.*\)", "")
#books['original_title'] = books['original_title'].str.replace(r"\(.*\)", "")

In [14]:
# step12: merge books and temp (temp's columns will be added twice with _x and_y)
books = books.merge(temp.drop_duplicates(['book_title']), how='left', left_on=['original_title'], right_on=['book_title'])
books = books.merge(temp.drop_duplicates(['book_title']), how='left', left_on=['title'], right_on=['book_title'])

In [15]:
# step13: add columns (snippet, pages, first_author) using columns from temp and drop all columns from temp
books['snippet'] = books['book_desc_y'].fillna(books['book_desc_x'])
books['pages'] = books['book_pages_y'].fillna(books['book_pages_x'])
books = books.drop(['book_desc_x', 'book_desc_y', 'book_pages_y', 'book_pages_x', 'book_rating_x', 'book_rating_y', 'book_title_y', 'book_title_x'], axis=1)
books['pages'] = books['pages'].fillna('0')
books['first_author'] = [x.split(',')[0] for x in books['authors']]
books['first_author'] = books['first_author'].fillna('Unknown')

In [16]:
# write finalbooks and finalratings csv files
os.chdir(cwd+'/output csv files')
books.to_csv("finalbooks.csv", index=False)
ratings.to_csv("finalratings.csv", index=False)

In [17]:
print(books.head(20))
print(ratings.head(20))

    book_id  goodreads_book_id  best_book_id   work_id  books_count  \
0         1            2767052       2767052   2792775          272   
1         2                  3             3   4640799          491   
2         3              41865         41865   3212258          226   
3         4               2657          2657   3275794          487   
4         5               4671          4671    245494         1356   
5         6           11870085      11870085  16827462          226   
6         7               5907          5907   1540236          969   
7         8               5107          5107   3036731          360   
8         9                960           960   3338963          311   
9        10               1885          1885   3060926         3455   
10       11              77203         77203   3295919          283   
11       12           13335037      13335037  13155899          210   
12       13               5470          5470    153313          995   
13    

In [18]:
# delete all dataframes used
del ratings, books, books2, book_tags, tags, temp